In [7]:
import json

# Sample dataset (replace with your actual dataset)
with open("../datasets/zeki_2_groceries_dataset_info_final.json") as f:
    dataset = json.load(f)

In [8]:
import os
import json
import ipywidgets as widgets
from IPython.display import display, clear_output

output_file = "../datasets/zeki_2_groceries_dataset_info_final.json"

# Widgets for displaying and editing data
image_widget = widgets.Image(format='jpg', layout=widgets.Layout(width='400px', height='300px'))

# Keep track of the items in the current entry
item_widgets = []

# Navigation buttons
prev_button = widgets.Button(description="Previous")
next_button = widgets.Button(description="Next")
close_button = widgets.Button(description="Close")
add_button = widgets.Button(description="Add Item")
delete_button = widgets.Button(description="Delete Entry", button_style="danger")  # For deleting the entire entry

# Label to display current index
index_label = widgets.Label(value="Current index: 0")

# Output display
output = widgets.Output()

# Index tracker for entries
entry_index = 0

def create_item_widgets(items):
    """
    Create a list of VBox widgets, each containing controls 
    for one item in the current entry, including a button 
    for deleting that item.
    """
    widgets_list = []
    for idx, item in enumerate(items):
        category = widgets.Text(value=item["category"], 
                                description=f"Category {idx}:")
        fine_grained = widgets.Text(value=item["fine-grained category"], 
                                    description=f"Fine-Grained {idx}:")
        count = widgets.IntText(value=item["count"], 
                                description=f"Count {idx}:")
        
        # Button to delete this single item from the entry
        delete_btn = widgets.Button(description="Delete", 
                                    button_style="danger")
        
        # Use a lambda that captures idx
        delete_btn.on_click(lambda b, item_idx=idx: delete_item(item_idx))
        
        widgets_list.append(
            widgets.VBox([category, fine_grained, count, delete_btn])
        )
    return widgets_list

def update_display():
    """
    Update the displayed widgets (image and item widgets) 
    based on the current entry_index.
    """
    global entry_index, item_widgets
    
    # If dataset is empty, clear everything
    if len(dataset) == 0:
        with output:
            clear_output()
            print("No entries left in the dataset.")
        image_widget.value = None
        index_label.value = "Current index: N/A"
        item_widgets.clear()
        return
    
    # Fetch the current entry
    entry = dataset[entry_index]
    
    # Update the index label
    index_label.value = f"Current index: {entry_index} / {len(dataset)-1}"
    
    # Update image widget
    image_path = os.path.join("..", entry["path"])
    if os.path.exists(image_path):
        with open(image_path, "rb") as f:
            image_widget.value = f.read()
    else:
        image_widget.value = None

    # Recreate item widgets for the current entry
    item_widgets.clear()
    item_widgets.extend(create_item_widgets(entry["items"]))

    # Update output display
    with output:
        clear_output()
        display(
            widgets.VBox([
                image_widget,
                widgets.VBox(item_widgets),
                controls
            ])
        )

def save_current_entry():
    """
    Save any changes made in the widget fields 
    (category, fine-grained, count) to the dataset 
    for the current entry, then write to file.
    """
    global entry_index, item_widgets
    
    if len(dataset) == 0:
        return  # Nothing to save
    
    entry = dataset[entry_index]
    # Apply widget values back to the dataset
    for idx, widget_group in enumerate(item_widgets):
        entry["items"][idx]["category"] = widget_group.children[0].value
        entry["items"][idx]["fine-grained category"] = widget_group.children[1].value
        entry["items"][idx]["count"] = widget_group.children[2].value
    
    # Save to JSON file
    with open(output_file, "w") as f:
        json.dump(dataset, f, indent=4)
    print(f"Changes saved for entry {entry_index}.")

def delete_item(idx):
    """
    Delete a single item from the current entry and update the file.
    Before deletion, we save any unsaved changes to the current entry.
    """
    global item_widgets, entry_index
    
    # First, save any unsaved changes in case user typed something
    save_current_entry()
    
    entry = dataset[entry_index]
    
    # Check if the index is valid
    if 0 <= idx < len(entry["items"]):
        del entry["items"][idx]
        
        # Save again to ensure the deletion is reflected
        with open(output_file, "w") as f:
            json.dump(dataset, f, indent=4)
        print(f"Item {idx} deleted and changes saved for entry {entry_index}.")
        
        # Update the display
        update_display()

def on_add_button_clicked(b):
    """
    Add an empty (new) item to the current entry.
    """
    global item_widgets, entry_index

    save_current_entry()
    
    entry = dataset[entry_index]
    prev_category = entry["items"][0]["category"]
    entry["items"].append({
        "category": prev_category,
        "fine-grained category": prev_category,
        "count": 1
    })

    # Save changes
    with open(output_file, "w") as f:
        json.dump(dataset, f, indent=4)
    print(f"Empty item added and changes saved for entry {entry_index}.")
    
    update_display()

def on_delete_entry_clicked(b):
    """
    Delete the entire current entry.
    1. Save current entry changes first.
    2. Remove the entry from the dataset.
    3. Adjust entry_index if necessary.
    4. Save dataset to file again.
    5. Update the display.
    """
    global entry_index, dataset
    
    if len(dataset) == 0:
        print("No entries to delete.")
        return
    
    # Save first
    save_current_entry()
    
    # Delete the entry from the dataset
    del dataset[entry_index]
    print(f"Entry {entry_index} deleted.")
    
    # If we deleted the last entry in the list, move index backward
    if entry_index >= len(dataset):
        entry_index = len(dataset) - 1
    
    # If the dataset is now empty, handle that
    if len(dataset) == 0:
        with open(output_file, "w") as f:
            json.dump(dataset, f, indent=4)
        update_display()
        return
    
    # Otherwise, save the updated dataset and refresh
    with open(output_file, "w") as f:
        json.dump(dataset, f, indent=4)
    
    update_display()

def on_next_button_clicked(b):
    """
    Go to the next entry. First, save the current entry changes.
    """
    global entry_index
    
    # Save any pending changes
    save_current_entry()
    
    # Move index if possible
    if entry_index < len(dataset) - 1:
        entry_index += 1
        update_display()

def on_prev_button_clicked(b):
    """
    Go to the previous entry. First, save the current entry changes.
    """
    global entry_index
    
    # Save any pending changes
    save_current_entry()
    
    # Move index if possible
    if entry_index > 0:
        entry_index -= 1
        update_display()

def on_close_button_clicked(b):
    """
    Clear the output area when closing.
    """
    with output:
        clear_output()

# Connect buttons to their event handlers
prev_button.on_click(on_prev_button_clicked)
next_button.on_click(on_next_button_clicked)
close_button.on_click(on_close_button_clicked)
add_button.on_click(on_add_button_clicked)
delete_button.on_click(on_delete_entry_clicked)  # <-- important for deleting entire entry

# Arrange widgets
controls = widgets.HBox([
    prev_button, 
    next_button, 
    close_button, 
    add_button, 
    delete_button, 
    index_label  # show the current index here
])

viewer = widgets.VBox([widgets.VBox(item_widgets), output])

# Finally, display the initial state
update_display()


Changes saved for entry 0.


Changes saved for entry 1.


Changes saved for entry 2.


Changes saved for entry 3.


Changes saved for entry 4.


...

Changes saved for entry 313.


Changes saved for entry 314.


Changes saved for entry 315.


Changes saved for entry 316.


Changes saved for entry 317.
Changes saved for entry 317.


In [9]:
# Initial display
update_display()

# Display the interface
display(viewer)

In [ ]:
sauces & condiments
horseradish
parmesan
canned & jarred goods
Curry Paste
Grated Cheese
seasoning
cornstarch
vegetable broth
chilli pepper
staple food